In [159]:
import requests
from bs4 import BeautifulSoup
import pymongo
import json
import pandas as pd
import time


In [168]:
# The default port used by MongoDB is 27017
# https://docs.mongodb.com/manual/reference/default-mongodb-port/
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

# Declare the database
db = client.jobs_db

# Declare the collection
collection = db.jobs

In [173]:
# search based on 7 days, salary, exact location of city, full time, and salary range from 80k

cities = ['Arlington','Washington']
state = ['VA','DC']
start = [1,51,101] #with limit set as 50 jobs per page, start refers to start count of jobs to scrape


url = 'https://www.indeed.com/jobs?q=Full+Time+%2480%2C000&l=Arlington%2C+VA&radius=0&sort=date&limit=50&fromage=7&start=51'


In [174]:
page = requests.get(url, headers=headers)
soup = BeautifulSoup(page.content, 'html.parser')
print(soup.title)

<title>Full Time $80,000 Jobs, Employment in Arlington, VA</title>


In [175]:
print(soup.prettify())

<!DOCTYPE html>
<html dir="ltr" lang="en">
 <head>
  <meta content="text/html;charset=utf-8" http-equiv="content-type"/>
  <script type="text/javascript">
   (function ( tk ) { if ( tk && document.images ) { var s="/", q="?", a="&", e="="; (new Image()).src = s+"rpc"+s+"log"+q+"a"+e+"jslat"+a+"tk"+e+tk; } })('1ev6595ijt5ai800');
  </script>
  <script src="//d3fw5vlhllyvee.cloudfront.net/s/ffe72ff/en_US.js" type="text/javascript">
  </script>
  <link href="//d3fw5vlhllyvee.cloudfront.net/s/105b986/jobsearch_all.css" rel="stylesheet" type="text/css"/>
  <link href="https://rss.indeed.com/rss?q=Full+Time+%2480%2C000&amp;l=Arlington%2C+VA&amp;radius=0&amp;sort=date" rel="alternate" title="Full Time $80,000 Jobs, Employment in Arlington, VA" type="application/rss+xml"/>
  <link href="/m/jobs?q=Full+Time+%2480%2C000&amp;l=Arlington%2C+VA&amp;radius=0&amp;sort=date&amp;limit=50&amp;fromage=7" media="only screen and (max-width: 640px)" rel="alternate"/>
  <script type="text/javascript">
   if 

In [176]:
# Retrieve the parent divs for all articles
   
results = soup.find_all('div', class_='jobsearch-SerpJobCard')
    
#loop over to get employment data
for result in results:
    #scrape jobs title
    jobs =  result.find(name="a", attrs={"data-tn-element":"jobTitle"}).text
    company = result.find(name='span', class_='company').text.strip()
    source = 'indeed.com'
    dates = div.find('span', class_= 'date').text
    #salary = result.find(name='span', class_='salaryText').text.strip()
    #location = result.find(name='span', class_='location').text
    

    #salary
    wages = result.find(name='span', class_='salaryText')
    if wages:
        salary=result.find(name='span', class_='salaryText').text.strip()
    else:
        salary = 'Not provided'

    #location
    locn = result.find(name='span', class_='location')
    if locn:
        location = result.find(name='span', class_='location').text
    else: 
        location = "Not provided" 
        
     

    #print employment data
    print('-----------------')
    print(jobs)
    print(company)
    print(salary)
    print(location)
    print(source)
    print(dates)
    
    # Dictionary to be inserted into MongoDB
    post = {
        'jobs' : jobs,
        'company' : company,
        'salary' : salary,
        'location': location,
        'source' : source,
        'date' : dates
    }
    
    # Insert dictionary into MongoDB as a document
    collection.update_one(
        {
            'jobs' : jobs,
            'company' : company,
            'salary' : salary,
            'location': location,
            'source' : source,
            'date' : dates
        },
        {'$set': {
            'jobs' : jobs,
            'company' : company,
            'salary' : salary,
            'location': location,
            'source' : source,
            'date' : dates
            }
        },
        upsert=True
    )

-----------------

Audi Service Advisor
Rosenthal Automotive
$55,000 - $110,000 a year
Not provided
indeed.com
Today
-----------------

GENERAL MANAGER/SERVICE MANAGER AUTOMOTIVE
AUTO STOP
$80,000 - $120,000 a year
Not provided
indeed.com
Today
-----------------

IT Service Operations Manager
Leidos
Not provided
Not provided
indeed.com
Today
-----------------

Data Engineer
BOEING
Not provided
Not provided
indeed.com
Today
-----------------

Vice President, Programs & Field Operations
National Older Worker Career Center
Not provided
Arlington, VA 22203 (Ballston-Virginia Square area)
indeed.com
Today
-----------------

ATC UAS Integration Analyst
Crown Consulting, Inc.
Not provided
Arlington, VA 22209 (North Rosslyn area)
indeed.com
Today
-----------------

Physician Assistant - FT/PT
American Executive Health System
$90,000 - $110,000 a year
Not provided
indeed.com
Today
-----------------

Military Readiness Analyst
Whitney, Bradley and Brown
Not provided
Arlington, VA
indeed.com
Toda